# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [2]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [3]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [4]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Data Modeling with Apache Cassandra for Sparkify Project

## The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The denormalized data should appear like below after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

#### Creating a Cluster

In [5]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [6]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify 
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1 }"""
)

except Exception as e:
    print(e)

#### Set Keyspace

In [7]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

#### Create Queries





#### Query 1

**Query Description:** In this query I used **sessionID** and **itemInSession** columns together as primary key, however partition key is **SessionId**.<br>

Before modeling, we must always think about queries first and then we must design our tables according to these queries.

In [8]:
## Query 1: List of the artist, song title and song's length in the music app history that was heard during where sessionId = 338 and itemInSession = 4
query1 = "CREATE TABLE IF NOT EXISTS session_history"
query1 = query1 + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY (sessionId, itemInSession))"
try:
    session.execute(query1)
except Exception as e:
    print(e)                   

In [9]:
# Inserting data into session_history table for Query 1

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query1 = "INSERT INTO session_history (sessionId, itemInSession, artist, song, length)"
        query1 = query1 + "VALUES (%s, %s, %s, %s, %s)"
        session.execute(query1, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

In [10]:
# Conversion of row data to dataframe to verify nosql queries for following sections.
def read_row_data():
    df = pd.read_csv(file)
    return df

#### Verifying Query 1

In [11]:
query1 = "select artist, song, length from session_history where sessionId = 338 and itemInSession = 4"

try:
    rows = session.execute(query1)
except Exception as e:
    print(e)

for row in rows:
    print("\033[1m"+"Artist:"+"\033[0m", row.artist, 
          "\033[1m"+"Song:"+"\033[0m", row.song, 
          "\033[1m"+"Length:"+"\033[0m", row.length
         )

Artist: Faithless Song: Music Matters (Mark Knight Dub) Length: 495.30731201171875


#### Verifying Query 1 by Comparing with DataFrame

In [12]:
df = read_row_data()
df = df[(df.sessionId==338) & (df.itemInSession==4)]
df = df[['artist', 'song', 'length', 'sessionId', 'itemInSession']]
df

,artist,song,length,sessionId,itemInSession
444,Faithless,Music Matters (Mark Knight Dub),495.3073,338,4


#### Query 2

**Query Description:** In this query I used **userId** and **SessionId** columns together as primary key, however partition key is **userId**.<br>

My clustering column is **itemInSession**. My data is sorted in ascending order since **itemInSession** is clustering column.

In [13]:
## Query 2: List the name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query2 = "CREATE TABLE IF NOT EXISTS user_history"
query2 = query2 + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY ((userId,SessionId),itemInSession))"
try:
    session.execute(query2)
except Exception as e:
    print(e)    

In [14]:
# Inserting data into user_history table for Query 2

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query2 = "INSERT INTO user_history (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query2 = query2 + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query2, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

#### Verifying Query 2

In [15]:
query2 = "select artist, song, firstName, lastName from user_history where userId = 10 and SessionId = 182"

try:
    rows = session.execute(query2)
except Exception as e:
    print(e)

for row in rows:
    print("\033[1m"+"Artist:"+"\033[0m", row.artist, 
          "\033[1m"+"Song:"+"\033[0m", row.song,
          "\033[1m"+"First Name:"+"\033[0m",row.firstname,
          "\033[1m"+"Last Name:"+"\033[0m", row.lastname
         )

Artist: Down To The Bone Song: Keep On Keepin' On First Name: Sylvie Last Name: Cruz
Artist: Three Drives Song: Greece 2000 First Name: Sylvie Last Name: Cruz
Artist: Sebastien Tellier Song: Kilometer First Name: Sylvie Last Name: Cruz
Artist: Lonnie Gordon Song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit) First Name: Sylvie Last Name: Cruz


#### Verifying Query 2 by Comparing with DataFrame

In [16]:
df = read_row_data()
df = df[(df.userId==10) & (df.sessionId==182)]
df = df[['artist', 'song', 'firstName', 'lastName', 'itemInSession']]
df

,artist,song,firstName,lastName,itemInSession
4704,Down To The Bone,Keep On Keepin' On,Sylvie,Cruz,0
4705,Three Drives,Greece 2000,Sylvie,Cruz,1
4706,Sebastien Tellier,Kilometer,Sylvie,Cruz,2
4707,Lonnie Gordon,Catch You Baby (Steve Pitron & Max Sanna Radio...,Sylvie,Cruz,3


#### Query 3

**Query Description:** In this query I used **song** and **userId** columns together as primary key, however partition key is **song** to list data fast.

In [17]:
## Query 3: List of every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query3 = "CREATE TABLE IF NOT EXISTS listen_history"
query3 = query3 + "(song text, userId int, firstName text, lastName text, PRIMARY KEY (song, userId))"
try:
    session.execute(query3)
except Exception as e:
    print(e)              

In [18]:
# Inserting data into listen_history table for Query 3

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query3 = "INSERT INTO listen_history (song, userId, firstName, lastName)"
        query3 = query3 + "VALUES (%s, %s, %s, %s)"
        session.execute(query3, (line[9], int(line[10]), line[1], line[4]))

#### Verifying Query 3

In [19]:
query3 = "select firstName, lastName from listen_history where song='All Hands Against His Own'"

try:
    rows = session.execute(query3)
except Exception as e:
    print(e)

for row in rows:
    print("\033[1m"+"First Name:"+"\033[0m",row.firstname,
          "\033[1m"+"Last Name:"+"\033[0m",row.lastname
         )

First Name: Jacqueline Last Name: Lynch
First Name: Tegan Last Name: Levine
First Name: Sara Last Name: Johnson


#### Verifying Query 3 by Comparing with DataFrame

In [20]:
df = read_row_data()
df = df[(df.song=='All Hands Against His Own')]
df = df[['userId', 'firstName', 'lastName', 'song']]
df

,userId,firstName,lastName,song
2792,80,Tegan,Levine,All Hands Against His Own
5135,95,Sara,Johnson,All Hands Against His Own
6298,29,Jacqueline,Lynch,All Hands Against His Own


### Drop the tables before closing out the sessions

In [21]:
try:
    query1 = "drop table if exists session_history"
    rows = session.execute(query1)
except Exception as e:
    print(e)

In [22]:
try:
    query2 = "drop table if exists user_history"
    rows = session.execute(query2)
except Exception as e:
    print(e)

In [23]:
try:
    query3 = "drop table if exists listen_history"
    rows = session.execute(query3)
except Exception as e:
    print(e)

### Close the session and cluster connection

In [24]:
session.shutdown()
cluster.shutdown()